In [ ]:
# import libraries
import keras
from utils.preprocess import *
from config import *

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# load dataset
dataset = load_unipen_dataset()

# data augmentation
augmentLayers = [
    # keras.layers.RandomRotation(0.01, fill_mode='constant'),
    keras.layers.RandomTranslation(0.01, 0.01, fill_mode='constant'),
    keras.layers.RandomZoom(0.01, fill_mode='constant'),
]

def augment(data, label):
    for layer in augmentLayers:
        data = layer(data)
    return data, label

dataset = dataset.shuffle(shuffle_buffer_size)
# dataset = dataset.repeat(2)
dataset = dataset.batch(batch_size)
dataset = dataset.map(augment)

# split & filter dataset
train_size = int(train_prop * dataset.cardinality().numpy())
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

In [ ]:
# build model
param_constraint = keras.constraints.MinMaxNorm(-2, 2, 1, 0)
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), name="conv1", input_shape=(32, 32, 1), kernel_constraint=param_constraint, bias_constraint=param_constraint),
    keras.layers.ReLU(name="relu1"),

    keras.layers.MaxPooling2D((2, 2), name="maxpool1"),

    keras.layers.Conv2D(48, (3, 3), name="conv2", kernel_constraint=param_constraint, bias_constraint=param_constraint),
    keras.layers.ReLU(name="relu2"),
    
    keras.layers.MaxPooling2D((2, 2), name="maxpool2"),
    
    keras.layers.Conv2D(80, (3, 3), name="conv3", kernel_constraint=param_constraint, bias_constraint=param_constraint),
    keras.layers.ReLU(name="relu3"),
    
    keras.layers.MaxPooling2D((2, 2), name="maxpool3"),
    # keras.layers.GlobalAveragePooling2D(name="avgpool"),

    keras.layers.Flatten(name="flatten"),
    keras.layers.Dropout(0.25),

    keras.layers.Dense(96, name="dense2", kernel_constraint=param_constraint, bias_constraint=param_constraint),
    keras.layers.ReLU(name="reludense2"),
    
    keras.layers.Dense(128 - 32, name="dense1", kernel_constraint=param_constraint, bias_constraint=param_constraint),
])

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(), # it says M2 is bad at latest Adam
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Param # = ((kersize ** 2) * in + 1) * out
model.summary()

In [ ]:
model.fit(train_dataset, epochs=epochs, validation_data=test_dataset)

In [ ]:
# evaluate model
test_loss, test_acc = model.evaluate(test_dataset)

print()
print('Test loss:    ', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
# save model
model.save(f"data/unipen_model.h5")